# Imports

In [1]:
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import libpysal.weights as weights
import pysal.explore as esda

c:\Users\Natha\Dev\irp\.venv\Lib\site-packages\spaghetti\network.py:41: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)
c:\Users\Natha\Dev\irp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set random seeds

In [67]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [68]:
raw_df = gpd.read_file("datasets/5_split/df_fe.gpkg")

In [69]:
df = raw_df.copy()

# Separate features

In [ ]:
# Drop R fold ids
df = df.drop(columns = [col for col in df.columns if "fold_id_r" in col])

# Dependent variables
labels = df.pop('very_good_health')

# Outer CV folds
outer_fold_ids = df["outer_loop_fold_id_python"]
outer_splits = np.sort(outer_fold_ids.unique().astype(int))

# Inner CV folds
inner_fold_ids = df[[col for col in df.columns if "inner_loop" in col]]
inner_splits = np.sort(inner_fold_ids.stack().unique().astype(int))

# Independent variables
features = df.drop(columns = [col for col in df.columns if "fold_id" in col])
features = features.drop(columns = ["geometry"])        # Need to add coords
predictor_cols = ["greenspace_proportion", "imd", "f_m_ratio", "mean_age"]

# Functions

## Get random hyperparameters

In [ ]:
def get_random_hyperparameters():
    no_of_layers = np.random.randint(1, 5)
    no_of_nodes = []
    for i in range(0, no_of_layers):
        no_of_nodes.append(np.random.randint((len(features.columns) / 2), (len(features.columns) * 2)))
    learning_rate = np.random.uniform(0.0001, 0.1)
    return no_of_layers, no_of_nodes, learning_rate
    

## Build model

In [72]:
def build_model(train_features, no_of_layers, no_of_nodes, learning_rate):

    layers = []

    normaliser = keras.layers.Normalization(axis = -1)
    normaliser.adapt(np.array(train_features))
    layers.append(normaliser)

    for layer_no in range(no_of_layers):
        layers.append(keras.layers.Dense(no_of_nodes[layer_no], activation = "relu"))

    layers.append(keras.layers.Dense(1))     # Single output for regression value

    model = keras.Sequential(
        layers
    )

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
        loss='mse'
    )

    return model

## Build early stopper

In [ ]:
def build_early_stopper():
    early_stopper = keras.callbacks.EarlyStopping(
        monitor = "val_loss",
        patience = 20,
        restore_best_weights = True
    )
    return early_stopper

## Get evaluation metrics

In [74]:
def get_evaluation_metrics(val_features, val_labels, predictions):
    mae = mean_absolute_error(val_labels, predictions)
    mse = mean_squared_error(val_labels, predictions)
    r2 = r2_score(val_labels, predictions)
    w = weights.KNN.from_dataframe(val_features, k = 8)
    moran = esda.esda.Moran(val_labels.values - predictions.flatten(), w)
    return mae, mse, r2, moran.I

## Get average score across cross-validation folds

In [75]:
def get_avg_scores(cv_results):
    mae_scores = []
    mse_scores = []
    r2_scores = []
    moran_scores = []

    for result in cv_results:
        mae_scores.append(result["mae"])
        mse_scores.append(result["mse"])
        r2_scores.append(result["r2"])
        moran_scores.append(result["moran"])

    avg_mae = np.mean(mae_scores)
    avg_mse = np.mean(mse_scores)
    avg_r2 = np.mean(r2_scores)
    avg_moran = np.mean(moran_scores)

    return avg_mae, avg_mse, avg_r2, avg_moran

## Get optimal hyperparameters

In [76]:
def get_optimal_hyperparameters(hp_combinations, cv_results):
    hp_combination_scores = []
    for i in range(len(hp_combinations)):
        current_hp_combination_results = [result for result in cv_results if result["hp_combination"] == i]
        mae, mse, r2, moran = get_avg_scores(current_hp_combination_results)
        hp_combination_scores.append(mse)
    optimal_combination = np.argmin(hp_combination_scores)
    optimal_hps = hp_combinations[optimal_combination]
    return optimal_hps

# Cross-validation

## Initialise HP and results arrays

In [2]:
outer_cv_results = []

##  Nested cross-validation loop

In [ ]:
for current_outer_split in outer_splits:

    hp_combinations = []
    cv_results = []

    # Get training and validation sets for current outer split
    is_in_validation_set = outer_fold_ids == current_outer_split
    is_in_training_set = ~is_in_validation_set
    outer_train_features = features.loc[is_in_training_set]
    outer_train_labels = labels.loc[is_in_training_set]
    outer_val_features = features.loc[is_in_validation_set]
    outer_val_labels = labels.loc[is_in_validation_set]
    current_inner_fold_ids = inner_fold_ids.loc[is_in_training_set]

    # Loop to test 10 hyperparameter combinations
    for i in range(8):
        
        # Get hyperparameters
        no_of_layers, no_of_nodes, learning_rate = get_random_hyperparameters()
        current_hps = {
            "outer_loop_split": current_outer_split,
            "no_of_layers": no_of_layers,
            "no_of_nodes": no_of_nodes,
            "learning_rate": learning_rate
        }
        hp_combinations.append(current_hps)

        # Inner cross-validation for model selection
        for current_inner_split in inner_splits:
            print(f"\n --- Training model {i} on outer split {current_outer_split}, inner split {current_inner_split} ---")

            # Get training and validation sets for current inner split
            is_in_validation_set = current_inner_fold_ids[f"inner_loop_{current_outer_split + 1}_fold_id_python"] == current_inner_split
            is_in_training_set = ~is_in_validation_set
            inner_train_features = outer_train_features.loc[is_in_training_set]
            inner_train_labels = outer_train_labels.loc[is_in_training_set]
            inner_val_features = outer_train_features.loc[is_in_validation_set]
            inner_val_labels = outer_train_labels.loc[is_in_validation_set]

            # Build model
            model = build_model(inner_train_features, no_of_layers, no_of_nodes, learning_rate)
            early_stopper = build_early_stopper()

            # Fit model
            model.fit(
                inner_train_features[predictor_cols],
                inner_train_labels,
                epochs = 200,
                validation_data = (inner_val_features, inner_val_labels),     # Research whether this is ok
                callbacks = [early_stopper],
                verbose = 0
            )

            # Get predictions using fitted model
            predictions = model.predict(inner_val_features).flatten()

            # Get accuracy scores
            mae, mse, r2, moran = get_evaluation_metrics(inner_val_labels, predictions)

            # Add scores for current fold to results
            cv_results.append({
                "hp_combination": i,
                "inner_split": current_inner_split,
                "hps": current_hps,
                "mae": mae,
                "mse": mse,
                "r2": r2,
                "moran": moran
            })

    # Get optimal hyperparameters for current outer split training set
    opt_hps = get_optimal_hyperparameters(hp_combinations, cv_results)
    opt_no_of_layers = opt_hps["no_of_layers"]
    opt_no_of_nodes = opt_hps["no_of_nodes"]
    opt_learning_rate = opt_hps["learning_rate"]

    # Build model
    model = build_model(outer_train_features, opt_no_of_layers, opt_no_of_nodes, opt_learning_rate)
    early_stopper = build_early_stopper()

    # Fit model
    model.fit(
        outer_train_features,
        outer_train_labels,
        epochs = 200,
        validation_data = (outer_val_features, outer_val_labels),     # Research whether this is ok
        callbacks = [early_stopper],
        verbose = 1
    )

    # Get predictions using fitted model
    predictions = model.predict(outer_val_features).flatten()

    # Get accuracy scores
    mae, mse, r2 = get_evaluation_metrics(outer_val_labels, predictions)

    # Add scores for current fold to results
    outer_cv_results.append({
        "outer_split": current_outer_split,
        "hps": opt_hps,
        "mae": mae,
        "mse": mse,
        "r2": r2,
        "moran": moran,
        "inner_cv_results": cv_results
    })

In [ ]:
with open("outputs/model_results/ann_fe.pkl", "wb") as f:
    pickle.dump(outer_cv_results, f)